I'll be using CNN articles as dataset. These articles are stored in multiple files along with their highlights. These highlight can be used to gauge the summary output. 

In [116]:
from tqdm.notebook import tqdm 

Let's try reading the file content 

In [117]:
def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

text = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
#text.split("\n")



It seems the we need to do some cleaning before we start working on the file.

In [118]:
text

'(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?\n\nSome scoff at the notion that movies do anything more than entertain. They are wrong. Sure, it\'s unlikely that one movie alone will change your views on issues of magnitude. But a movie (or TV show) can begin your "education" or "miseducation" on a topic. And for those already agreeing with the film\'s thesis, it can further entrench your views.\n\nAnyone who doubts the potential influence that movies can have on public opinion need to look no further than two films that are causing an uproar even before they have opened nationwide. They present hot button issues that manage to fire up people from the left and right.\n\nThe first, "Zero Dark Thirty," is about the pursuit and killing of Osama bin Laden, which feature

In [119]:
import re
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
en_stopwords = stopwords.words('english')

def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    # cnn_indx = text.find("(CNN)")
    # if cnn_indx >=0:
    #     text = text[cnn_indx+len("(CNN)"):]
        
    #text = re.sub("\n"," ",text).lower()

    #text.split("\n")
    # cleaned = list()
    # table = str.maketrans('','',string.punctuation)

    # for line in text:
       
    #     line = line.split(" ")

    #     line = [w.translate(table) for w in line]
        
    #     line = [w for w in line if w.isalpha()]
    #     cleaned.append(' '.join(line))

    # text = [sent for sent in cleaned if len(sent)>0]

    return text

article = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
dmarticle = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/dailymail/stories/0a00a9aebcb754c51534867cf1db2335dcb76884.story')
# t = article.split("\n")
# cleaned = list()
# table = str.maketrans('','',string.punctuation)
# for w in t:
#     if len(w)>0:
#         w = w.split(" ")
#         w = [word.translate(table) for word in w]
#         w = [word for word in w if word.isalpha()]
#         cleaned.append(" ".join(w))
# print(cleaned)
#print(dmarticle)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rohankilledar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now we need to split the article and the hightlights

In [120]:
def split_text(article):
    indx = article.index('@highlight')
    story = article[:indx]
    highlight = article[indx:].split('@highlight')

    highlight = ". ".join([h.strip() for h in highlight if len(h)>0])
    return story,highlight

#st,hi = split_text(dmarticle)
#print("story: "+st)
#print("highlight: "+hi)

In [121]:
#st.replace("\n\n"," ").replace("\xa0", "")


one of the dataset file seems to be usable now. Need to load all of the stories together using these given functions

In [122]:
from os import listdir

def read_all(folder):
    dataset = list()

    for file in tqdm(listdir(folder)):
        filename = folder + '/' + file
        article = read_file(filename)
        story,highlight = split_text(article)

        dataset.append({'story':story, 'highlight':highlight})
    
    return dataset

folder = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories"
daily_mail = '/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/dailymail/stories'

dataset = read_all(folder)
daily_mail_dataset = read_all(daily_mail)
dataset.extend(daily_mail_dataset)

print("total stories in dataset: "+ str(len(dataset)))

  0%|          | 0/92579 [00:00<?, ?it/s]

  0%|          | 0/219506 [00:00<?, ?it/s]

total stories in dataset: 312085


In [123]:
import pickle
output_file = open('unprocessed_dataset.pkl','wb')
pickle.dump(dataset, output_file)
output_file.close()

In [124]:
del dataset[53734] # this story contained 100 digit long decimal pointer of pi value and was causing issue while converting the number to its text form

Trying different text preprocessing on our dataset.

In [125]:
import inflect
p = inflect.engine()
p.number_to_words(1234567)


example = dataset[1]['story'].replace("\n\n"," ").replace("\xa0", "").replace('-',' ')
#print(example)
lines = example.split('. ')
table = str.maketrans('', '', string.punctuation)
for line in lines:
    token = line.split()
    #print(token)
    token =  [word.lower() if not word.isnumeric() else p.number_to_words(eval(word)) for word in token]
    #print(token)
    token = [w.translate(table) for w in token]
    #print(token)
    # answer = list()
    # for word in token:
    #     if not word.isnumeric():
    #         answer.append(word)
    #     else:
    #         answer.append(p.number_to_words(int(word)))

    answer = [p.number_to_words(int(word)) if isinstance(word,int) else word for word in token ]
    #print(answer)
    #token = [p.number_to_words(word) for word in token if not word.isalpha()]
    #removing the non-characters
    sentence = ' '.join(answer).replace('[^A-za-z\s]+', '')
    #normalize
    #sentence = sentence.normalize('NFD')
    #convert to ASCII from unicode
    sentence = sentence.encode('ascii',errors='ignore').decode('utf-8')
    print(sentence)


cnngeorge zimmerman acquitted by a florida jury over the death of trayvon martin was arrested in florida on suspicion of aggravated assault and domestic violence with a weapon local authorities said
the thirtyone year old florida resident was arrested friday by police in lake mary at about ten pm
and booked into the john e
polk correctional facility according to that facilitys website
that facility like its website is run by the seminole county sheriffs office
it all came about after zimmerman allegedly threw a wine bottle at a girlfriend his lawyer don west told reporters
whatever happened took place several days ago said west
and as far as i know they have not been together for some time certainly not since then police first learned about it after coming in contact with the alleged victim at a traffic stop on monday lake mary police spokeswoman bianca gillett said
when asked about the delay in making the arrest gillett said the arresting officer tried to reach zimmerman on monday and

Data Cleaning

In [126]:
import re
import inflect
p = inflect.engine()

def clean_data(data):
    cleaned = list()
    table = str.maketrans('', '', string.punctuation)
    for line in data:
        #line = re.split(r"(CNN)+ --",line)

        dash_indx = line.find("--")
        if dash_indx >=0:
            line = line[dash_indx+len("--"):]
        
        line = line.replace('-',' ')

        #line = line.trim()
        # tokenize on white space
        line = line.split()
        # convert to lower case and replacing - with space for words like 31-year-old
        #line = [word.lower() for word in line]
        line = [word.lower() if not word.isdigit() else p.number_to_words(word) for word in line]
        #line = [p.number_to_words(int(word)) if isinstance(word,int) else word for word in line ]
        #remove all non-characters
        #line =[word.replace('[^A-Za-z\s]+', '') for word in line]
        # remove punctuation from each token
        line = [w.translate(table) for w in line]
        # remove tokens with numbers in them instead of removing the nisumbers we can convert them into their spelling using inflect library
        #line = [word for word in line if word.isalpha()]
        # answer = list()
        # for word in line:
        #     if not isinstance(word,int):
        #         answer.append(word)
        #     else :
        #         answer.append(p.number_to_words(int(word)))

        # store as string and removing non-characters
        cleaned.append(' '.join(line))
	# remove empty strings
    cleaned = [c for c in cleaned if len(c) > 0]
    return cleaned


#dataset[0]['story'] = clean_data(dataset[0]['story'].split('\n'))
#dataset[0]['highlight'] = clean_data(dataset[0]['highlight'])
#     #text = re.sub("\n"," ",text).lower()
# cleaned_dataset = dataset.copy()

    

In [127]:
for i in tqdm(range(len(dataset))):

    dataset[i]['story']= '. '.join(clean_data(dataset[i]['story'].split('\n')))#.lstrip("[\'").rstrip("\']")
    dataset[i]['highlight'] = ''.join(clean_data(dataset[i]['highlight'].split('\n')))#.lstrip("[\'").rstrip("\']")


  0%|          | 0/312084 [00:00<?, ?it/s]

In [128]:
dataset[1]['story']

'acquitted by a florida jury over the death of trayvon martin was arrested in florida on suspicion of aggravated assault and domestic violence with a weapon local authorities said. the thirtyone year old florida resident was arrested friday by police in lake mary at about ten pm and booked into the john e polk correctional facility according to that facilitys website that facility like its website is run by the seminole county sheriffs office. it all came about after zimmerman allegedly threw a wine bottle at a girlfriend his lawyer don west told reporters. whatever happened took place several days ago said west and as far as i know they have not been together for some time certainly not since then. police first learned about it after coming in contact with the alleged victim at a traffic stop on monday lake mary police spokeswoman bianca gillett said. when asked about the delay in making the arrest gillett said the arresting officer tried to reach zimmerman on monday and tuesday but c

Saving the dataset

In [129]:
import csv
csv_columns = ['story','highlight']
csv_file = "data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dataset:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [131]:
import pandas as pd

dataset_pd = pd.read_csv("data.csv")

In [32]:
#dataset_pd['highlight'][0].lstrip("[\'").rstrip("\']")
dataset_pd['story'][0].lstrip("[\'").rstrip("\']")

'at the start of a big week for the higgs boson the most soughtafter particle in all of physics scientists in illinois said monday that they had crept closer to proving that the particle exists but had been unable to reach a definitive conclusion. the scientists outlined their final analysis based on more than years of research and trillion particle collisions using the us department of energys fermilab tevatron collider near batavia illinois whose budgetary woes shut it down last year. what is the higgs boson and why is it important. their announcement came two days before researchers at the large hadron collider under the alps are due to unveil their latest results at an eagerly awaited seminar at the cern particle physics laboratory in geneva switzerland. our data strongly point toward the existence of the higgs boson rob roser a spokesman for one of two independent experiments at the tevatron said in a statement but it will take results from the experiments at the large hadron coll

In [132]:
import pickle
output_file = open('dataset.pkl','wb')
pickle.dump(dataset_pd, output_file)
output_file.close()

In [34]:
#dataset_pd['highlight'] = dataset_pd['highlight'].apply(lambda x:  'sostok ' + x.lstrip("[\'").rstrip("\']") + ' eostok')
#dataset_pd['story'] = dataset_pd['story'].apply(lambda x: x.lstrip("[\'").rstrip("\']") )


In [133]:
#dataset_pd.drop('hightlight1', axis=1, inplace=True)
dataset_pd['word_count_text'] = dataset_pd['story'].apply(lambda x: len(str(x).split()))

In [ ]:
import numpy as np
for i in range(0,100,10) :
    var =dataset_pd["word_count_text"]. values
    var = np.sort(var,axis = None)
    print (" {} percentile value is {}".format (i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

 0 percentile value is 1
 10 percentile value is 293
 20 percentile value is 379
 30 percentile value is 451
 40 percentile value is 521
 50 percentile value is 592
 60 percentile value is 672
 70 percentile value is 766
 80 percentile value is 891
 90 percentile value is 1096
100 percentile value is  2094


In [134]:
for i in range (90,100) :
    var =dataset_pd["word_count_text"].values
    var = np.sort(var, axis = None)
    print("{} percentile value is {}".format(i, var[int (len(var)*(float (i)/100))]))
print ("100 percentile value is ", var[-1])

NameError: name 'np' is not defined